<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-02 14:31:48
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.35 C
-------------------
Today PnL: -1.08 L (-0.79%)
Current PnL: -34.05 L (-21.71%)
CY Booked + Current PnL: -19.31 L (-12.31%)
-------------------
Total profit:  81.21 K
Total loss:  -34.86 L
-------------------
Total Booked + Current PnL: 7.67 L (5.91%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.88%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 99.55 L (73.5%)
Deployed:  1.30 C
Current:  1.35 C
CAGR/XIRR %: 2.21%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-5.09,2.86,15.90,19.23,24609.0,4310.0,154771.0,484.83,21.35,49.0,L-LC,12.46,182.0,0.18,1.14,24.75,XY25,ATH,MINING
77,TTKPRESTIG,-2.25,-26.56,36.28,0.09,26853.0,-26762.0,74015.0,770.00,71.82,33.0,M-SC,6.50,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES
50,MASFIN,-4.66,-6.31,30.26,22.05,27779.0,-6180.0,91800.0,398.61,-19.15,45.0,H-SC,3.42,168.0,-0.22,0.68,33.44,XR,ATH,FINANCE
33,HCLTECH,-2.49,8.94,13.67,23.83,36028.0,21632.0,263556.0,1908.19,13.61,52.0,X-LC,7.31,13.0,0.60,1.95,26.45,X40,ATH,IT
51,MEDANTA,8.82,-6.04,32.05,24.08,38230.0,-7668.0,119282.0,1486.00,-13.67,50.0,X-SC,9.37,91.0,-0.20,0.88,9.25,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,MEDANTA,8.82,-6.04,32.05,24.08,38230.0,-7668.0,119282.0,1486.0,-13.67,50.0,X-SC,9.37,91.0,-0.20,0.88,9.25,XY24,NTT,HEALTHCARE
15,CAMPUS,6.44,-10.64,49.15,33.28,73692.0,-17843.0,149932.0,393.0,-28.06,57.0,M-SC,5.70,221.0,-0.24,1.11,16.21,XY24,NTT,FOOTWEAR
7,ATULAUTO,6.02,-26.95,99.27,45.57,142956.0,-53125.0,144007.0,844.0,3105.41,50.0,M-SC,8.33,245.0,-0.37,1.06,8.98,XY24,NTT,AUTO
70,SYMPHONY,5.72,-28.71,40.23,-0.03,49081.0,-49141.0,122000.0,1306.0,-42.85,63.0,M-SC,12.88,206.0,-1.00,0.90,12.50,OX40N,NTT,DURABLES
68,STARHEALTH,4.46,-8.93,61.71,47.26,157981.0,-25117.0,256006.0,761.0,24.31,64.0,H-SC,3.33,174.0,-0.16,1.89,37.16,XY24,NTT,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,-6.58,-37.83,81.60,12.90,110850.0,-82651.0,135845.0,593.00,100.72,41.0,H-SC,6.24,173.0,-0.75,1.00,12.95,OX40N,NTT,FINANCE
5,ANGELONE,-6.52,-2.73,27.69,24.20,82215.0,-8338.0,296912.0,3033.00,38.75,41.0,X-SC,2.37,99.0,-0.10,2.19,24.35,X40N,NTT,FINANCE
29,GREENPANEL,-5.18,-42.10,147.20,43.14,156044.0,-77070.0,106008.0,537.00,137.73,45.0,M-SC,17.35,231.0,-0.49,0.78,3.66,XY24,NTT,MISC
74,TITAGARH,-5.10,-26.65,98.69,45.74,175308.0,-64542.0,177635.0,1548.00,-2.22,42.0,H-SC,5.79,175.0,-0.37,1.31,15.53,XY24,NTT,ENGINEERING
18,COALINDIA,-5.09,2.86,15.90,19.23,24609.0,4310.0,154771.0,484.83,21.35,49.0,L-LC,12.46,182.0,0.18,1.14,24.75,XY25,ATH,MINING


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-1.01,-0.59,46.42,45.57,120733.0,-1531.0,260088.0,7934.0,-36.11,65.0,H-MC,6.88,121.0,-0.01,1.92,15.50,AR,NTT,ELECTRICAL
84,WIPRO,2.13,-0.76,73.59,72.27,123745.0,-1293.0,168155.0,420.0,-14.70,46.0,M-LC,2.57,101.0,-0.01,1.24,8.09,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.63,-9.41,129.51,107.91,173222.0,-13892.0,133752.0,1641.55,-26.23,39.0,M-SC,10.80,220.0,-0.08,0.99,0.63,OX40N,ATH,CABLES
72,TATAELXSI,1.59,-22.06,69.40,32.03,71310.0,-29083.0,102752.0,9161.00,-15.70,50.0,H-SC,6.51,157.0,-0.41,0.76,15.09,OX40N,NTT,IT
48,KANSAINER,3.42,-22.20,45.85,13.47,96197.0,-59859.0,209808.0,340.00,-67.92,53.0,H-SC,5.30,159.0,-0.62,1.55,6.68,XY24,NTT,PAINTS
66,SIS,2.83,-22.89,57.26,21.26,49024.0,-25416.0,85616.0,528.00,2021.76,55.0,H-SC,4.81,163.0,-0.52,0.63,15.68,OX40N,NTT,MISC
77,TTKPRESTIG,-2.25,-26.56,36.28,0.09,26853.0,-26762.0,74015.0,770.00,71.82,33.0,M-SC,6.50,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-5.09,2.86,15.9,19.23,24609.0,4310.0,154771.0,484.83,21.35,49.0,L-LC,12.46,182.0,0.18,1.14,24.75,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-1.01,-0.59,46.42,45.57,120733.0,-1531.0,260088.0,7934.00,-36.11,65.0,H-MC,6.88,121.0,-0.01,1.92,15.50,AR,NTT,ELECTRICAL
84,WIPRO,2.13,-0.76,73.59,72.27,123745.0,-1293.0,168155.0,420.00,-14.70,46.0,M-LC,2.57,101.0,-0.01,1.24,8.09,XR,NTT,IT
39,INDIAMART,0.98,-5.61,119.00,106.72,138540.0,-6916.0,116420.0,4810.62,-53.30,49.0,H-SC,9.05,138.0,-0.05,0.86,16.46,AR,ATH,MISC
26,FINCABLES,0.63,-9.41,129.51,107.91,173222.0,-13892.0,133752.0,1641.55,-26.23,39.0,M-SC,10.80,220.0,-0.08,0.99,0.63,OX40N,ATH,CABLES
85,ZYDUSLIFE,-2.01,-7.96,47.36,35.63,91352.0,-16685.0,192889.0,1286.17,-20.73,38.0,H-MC,5.48,119.0,-0.18,1.42,7.38,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JIOFIN,-4.83,-22.09,59.79,24.48,124538.0,-59073.0,208292.0,387.00,-23.43,18.0,X-LC,7.62,37.0,-0.47,1.54,20.73,XY24,BTT,FINANCE
44,ITC,-2.65,-23.14,44.13,10.78,81513.0,-55608.0,184710.0,452.00,-54.23,20.0,X-LC,18.19,1.0,-0.68,1.36,0.00,X40,NTT,FMCG
12,BATAINDIA,0.05,-44.76,144.75,35.20,102890.0,-57589.0,71081.0,2096.00,-6.86,21.0,X-SC,19.87,93.0,-0.56,0.52,1.01,X40,NTT,FOOTWEAR
13,BERGEPAINT,0.43,-17.18,46.27,21.14,87119.0,-39055.0,188284.0,680.00,-22.97,21.0,X-MC,7.28,75.0,-0.45,1.39,0.43,XY24,NTT,PAINTS
6,ASIANPAINT,-1.30,-16.23,44.38,20.95,93602.0,-40858.0,210910.0,3460.25,-17.08,23.0,X-LC,10.58,36.0,-0.44,1.56,13.62,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,1.19,-13.09,36.40,18.55,119663.0,-49493.0,328744.0,4311.59,-25.24,50.0,X-LC,0.55,3.0,-0.41,2.43,11.84,X40,ATH,IT
81,VBL,-1.47,-7.04,45.95,35.67,134913.0,-22234.0,293608.0,671.64,-18.11,35.0,X-LC,1.79,5.0,-0.16,2.17,5.84,X40N,ATH,FMCG
34,HINDUNILVR,-1.16,-8.76,24.58,13.67,57652.0,-22513.0,234550.0,2922.00,-15.61,46.0,X-LC,2.23,9.0,-0.39,1.73,10.73,XY25,NTT,FMCG
5,ANGELONE,-6.52,-2.73,27.69,24.20,82215.0,-8338.0,296912.0,3033.00,38.75,41.0,X-SC,2.37,99.0,-0.10,2.19,24.35,X40N,NTT,FINANCE
36,ICICIGI,-0.46,-3.74,23.98,19.34,46191.0,-7486.0,192623.0,2252.93,-24.04,34.0,X-MC,2.75,71.0,-0.16,1.42,11.60,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,-3.10,-31.65,140.15,64.15,227954.0,-75301.0,162650.0,3906.00,-59.81,31.0,X-SC,7.24,84.0,-0.33,1.20,0.00,XY24,BTT,CEMENT
32,HAVELLS,-1.55,-19.00,63.06,32.08,167532.0,-62320.0,265671.0,2062.85,-23.13,26.0,X-MC,7.85,69.0,-0.37,1.96,0.00,X40,ATH,ELECTRICAL
44,ITC,-2.65,-23.14,44.13,10.78,81513.0,-55608.0,184710.0,452.00,-54.23,20.0,X-LC,18.19,1.0,-0.68,1.36,0.00,X40,NTT,FMCG
10,BAJAJHLDNG,-1.46,-5.49,36.72,29.22,65982.0,-10435.0,179690.0,14451.52,-8.00,37.0,X-LC,13.83,29.0,-0.16,1.33,0.09,X40,ATH,FINANCE
53,PGHH,0.20,-11.75,52.06,34.19,98098.0,-25088.0,188432.0,17907.65,-38.01,28.0,X-MC,7.35,60.0,-0.26,1.39,0.20,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,1.19,-30.07,104.27,42.84,47399.0,-19548.0,45458.0,424.00,-54.27,51.0,X-SC,21.54,83.0,-0.41,0.34,6.85,XY24,NTT,MISC
12,BATAINDIA,0.05,-44.76,144.75,35.20,102890.0,-57589.0,71081.0,2096.00,-6.86,21.0,X-SC,19.87,93.0,-0.56,0.52,1.01,X40,NTT,FOOTWEAR
58,RELAXO,-4.15,-51.44,213.64,52.31,150597.0,-74669.0,70491.0,1176.00,-48.31,44.0,X-SC,11.46,92.0,-0.50,0.52,4.82,X40N,NTT,FOOTWEAR
51,MEDANTA,8.82,-6.04,32.05,24.08,38230.0,-7668.0,119282.0,1486.00,-13.67,50.0,X-SC,9.37,91.0,-0.20,0.88,9.25,XY24,NTT,HEALTHCARE
35,HONAUT,3.87,-17.62,73.97,43.32,99253.0,-28692.0,134180.0,58357.33,-29.17,53.0,X-SC,8.90,90.0,-0.29,0.99,8.95,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-2.65,-23.14,44.13,10.78,81513.0,-55608.0,184710.0,452.00,-54.23,20.0,X-LC,18.19,1.0,-0.68,1.36,0.00,X40,NTT,FMCG
75,TMPV,3.39,-25.82,65.79,22.98,110238.0,-58338.0,167560.0,600.00,-23.08,58.0,X-LC,4.46,2.0,-0.53,1.24,7.12,XY24,NTT,AUTO
73,TCS,1.19,-13.09,36.40,18.55,119663.0,-49493.0,328744.0,4311.59,-25.24,50.0,X-LC,0.55,3.0,-0.41,2.43,11.84,X40,ATH,IT
81,VBL,-1.47,-7.04,45.95,35.67,134913.0,-22234.0,293608.0,671.64,-18.11,35.0,X-LC,1.79,5.0,-0.16,2.17,5.84,X40N,ATH,FMCG
42,INFY,-1.17,10.40,21.59,34.24,74581.0,32555.0,345443.0,1972.00,-13.85,46.0,X-LC,7.16,6.0,0.44,2.55,19.55,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.46,-33.98,101.27,32.89,51622.0,-26232.0,50975.0,1800.00,-729.38,50.0,L-MC,11.29,259.0,-0.51,0.38,40.38,XR,NTT,BANKS
14,BSOFT,3.32,-23.22,96.39,50.79,104093.0,-32661.0,107992.0,831.70,-1.43,53.0,H-SC,5.94,171.0,-0.31,0.80,25.74,XR,ATH,IT
18,COALINDIA,-5.09,2.86,15.90,19.23,24609.0,4310.0,154771.0,484.83,21.35,49.0,L-LC,12.46,182.0,0.18,1.14,24.75,XY25,ATH,MINING
33,HCLTECH,-2.49,8.94,13.67,23.83,36028.0,21632.0,263556.0,1908.19,13.61,52.0,X-LC,7.31,13.0,0.60,1.95,26.45,X40,ATH,IT
42,INFY,-1.17,10.40,21.59,34.24,74581.0,32555.0,345443.0,1972.00,-13.85,46.0,X-LC,7.16,6.0,0.44,2.55,19.55,X40,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SATIN,-0.86,-15.24,79.14,51.83,185319.0,-42118.0,234166.0,274.00,-23.78,57.0,H-SC,3.13,148.0,-0.23,1.73,14.07,XY24,NTT,FINANCE
27,GILLETTE,-1.41,2.50,29.18,32.42,75941.0,6354.0,260250.0,11206.78,-15.80,63.0,X-SC,12.66,87.0,0.08,1.92,17.98,X40,ATH,FMCG
1,ABB,-1.01,-0.59,46.42,45.57,120733.0,-1531.0,260088.0,7934.00,-36.11,65.0,H-MC,6.88,121.0,-0.01,1.92,15.50,AR,NTT,ELECTRICAL
33,HCLTECH,-2.49,8.94,13.67,23.83,36028.0,21632.0,263556.0,1908.19,13.61,52.0,X-LC,7.31,13.0,0.60,1.95,26.45,X40,ATH,IT
68,STARHEALTH,4.46,-8.93,61.71,47.26,157981.0,-25117.0,256006.0,761.00,24.31,64.0,H-SC,3.33,174.0,-0.16,1.89,37.16,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.77
1,25,44.76
2,50,76.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.10
MC    30.57
LC    24.32
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.42
X40      23.99
X40N     14.88
AR        8.88
XR        8.73
XY25      8.66
OX40N     7.51
SR        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.94
H-SC    22.97
X-LC    20.80
M-SC    11.15
X-SC    10.23
H-MC     4.79
M-MC     1.46
M-LC     1.24
L-LC     1.14
H-LC     1.14
L-SC     0.75
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.20,-14.98,54.17
FINANCE,13.07,-18.07,64.56
IT,12.95,-21.90,82.80
MISC,6.91,-38.74,95.49
ELECTRICAL,6.01,-15.35,57.44
PAINTS,5.44,-25.31,43.93
INSURANCE,4.85,-3.26,38.60
PHARMA,4.03,-7.52,41.36
AUTO,2.94,-31.69,77.23


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3393034.0,21
AR,1369523.0,10
XR,1311869.0,12
X40,1259577.0,15
X40N,1104633.0,10
OX40N,775202.0,10
XY25,446449.0,6
SR,295008.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3682595.0,24
X-MC,1787403.0,16
M-SC,1550291.0,15
X-LC,1076637.0,12
X-SC,911280.0,9
H-MC,429140.0,3
L-SC,190195.0,2
M-LC,123745.0,1
H-LC,79824.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1258249.0      6
           AR         947658.0      5
M-SC       XY24       880506.0      6
H-SC       XR         847776.0      7
X-MC       X40        609377.0      7
           XY24       488865.0      3
X-LC       X40        471369.0      6
X-MC       X40N       446399.0      4
X-SC       X40N       418866.0      4
M-SC       OX40N      346931.0      5
H-SC       OX40N      333904.0      4
X-SC       XY24       313583.0      3
H-SC       SR         295008.0      2
X-MC       XY25       242762.0      2
X-LC       X40N       239368.0      2
           XY24       234776.0      2
H-MC       XY24       217055.0      1
           AR         212085.0      2
M-SC       XR         192898.0      2
X-SC       X40        178831.0      2
X-LC       XY25       131124.0      2
M-SC       AR         129956.0      2
M-LC       XR         123745.0      1
L-SC       XR          95828.0      1
           OX40N       94367.0      1
H-LC       AR          79824.0      1
L-MC       XR          51622.0      1
M-MC       XY25        47954.0      1
L-LC       XY25        24609.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
